# Quickstart notebook
The example code here shows how to get up and running with Mosaic using the Python API.

In [0]:
from pyspark.sql.functions import *

## Enable Mosaic in the notebook
To get started, you'll need to attach the python library to your cluster and execute the `enable_mosaic` function.

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)

/databricks/spark/python/pyspark/sql/context.py:82: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(

Mosaic has extra configuration options. Check the docs for more details.

In [0]:
help(enable_mosaic)

Help on function enable_mosaic in module mosaic.api.enable:

enable_mosaic(spark: pyspark.sql.session.SparkSession, dbutils=None) -> None
 Enable Mosaic functions.
 
 Use this function at the start of your workflow to ensure all of the required dependencies are installed and
 Mosaic is configured according to your needs.
 
 Parameters
 ----------
 spark : pyspark.sql.SparkSession
 The active SparkSession.
 dbutils : dbruntime.dbutils.DBUtils
 The dbutils object used for `display` and `displayHTML` functions.
 Optional, only applicable to Databricks users.
 
 Returns
 -------
 
 Notes
 -----
 Users can control various aspects of Mosaic's operation with the following Spark confs:
 
 - `spark.databricks.mosaic.jar.autoattach`: 'true' (default) or 'false'
 Automatically attach the Mosaic JAR to the Databricks cluster? (Optional)
 - `spark.databricks.mosaic.jar.location`
 Explicitly specify the path to the Mosaic JAR.
 (Optional and not required at all in a standard Databricks environment).
 - `spark.databricks.mosaic.geometry.api`: 'OGC' (default) or 'JTS'
 Explicitly specify the underlying geometry library to use for spatial operations. (Optional)
 - `spark.databricks.mosaic.index.system`: 'H3' (default)
 Explicitly specify the index system to use for optimized spatial joins. (Optional)

## Geometry constructors and the Mosaic internal geometry format

Mosaic allows users to create new Point geometries from a pair of Spark DoubleType columns.

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

+----+-----+--------------------+
 lat| lon| point_geom|
+----+-----+--------------------+
35.0|-80.0|{1, [[[-80.0, 35....|
45.0|-80.0|{1, [[[-80.0, 45....|
45.0|-70.0|{1, [[[-70.0, 45....|
35.0|-70.0|{1, [[[-70.0, 35....|
35.0|-80.0|{1, [[[-80.0, 35....|
+----+-----+--------------------+

Mosaic Point geometries can be aggregated into LineString and Polygon geometries using the respective constructors.

In [0]:
from mosaic import st_makeline

bounds_df = (
  bounds_df
  .groupBy()
  .agg(collect_list("point_geom").alias("bounding_coords"))
  .select(st_makeline("bounding_coords").alias("bounding_ring"))
)
bounds_df.show()

+--------------------+
 bounding_ring|
+--------------------+
{3, [[[-80.0, 35....|
+--------------------+

In [0]:
from mosaic import st_makepolygon

bounds_df = bounds_df.select(st_makepolygon("bounding_ring").alias("bounds"))
bounds_df.show()

+--------------------+
 bounds|
+--------------------+
{5, [[[-80.0, 35....|
+--------------------+

## Geometry clipping without an index

Mosaic implements set intersection functions: contains, intersects, overlaps etc. Here you can see `st_contains` being used to clip points by a polygon geometry.

In [0]:
tripsTable = spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")

In [0]:
from mosaic import st_contains
trips = (
  tripsTable
  .limit(5_000_000)
  .repartition(sc.defaultParallelism * 20)
  .drop("vendorId", "rateCodeId", "store_and_fwd_flag", "payment_type")
  .withColumn("pickup_geom", st_point("pickup_longitude", "pickup_latitude"))
  .withColumn("dropoff_geom", st_point("dropoff_longitude", "dropoff_latitude"))
  .crossJoin(bounds_df)
  .where(st_contains("bounds", "pickup_geom"))
  .where(st_contains("bounds", "dropoff_geom"))
  .cache()
)

In [0]:
trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
 VTS|2009-04-23 22:34:00|2009-04-23 22:52:00| 1| 4.45| -73.994352| 40.70314| null| -73.998247| 40.745143| 13.7| 0.5| null| 0.0| 0.0| 14.2|{1, [[[-73.994352...|{1, [[[-73.998247...|{5, [[[-80.0, 35....|
 VTS|2009-07-15 18:02:00|2009-07-15 18:15:00| 5| 2.65| -74.000485| 40.732423| null| -73.970637| 40.75256| 9.7| 1.0| null| 0.0| 0.0| 10.7|{1, [[[-74.000485...|{1, [[[-73.970637...|{5, [[[-80.0, 35....|
 CMT|2010-02-11 19:50:32|2010-02-11 19:58:20| 1| 1.3| -74.015856| 40.711017| 1| -74.007158| 40.726024| 6.9| 1.0| 0.5| 0.0| 0.0| 8.4|{1, [[[-74.015856...|{1, [[[-74.007158...|{5, [[[-80.0, 35....|
 VTS|2009-09-17 05:51:00|2009-09-17 05:54:00| 1| 0.78| -74.002052| 40.72806| null| -73.996152| 40.737065| 4.1| 0.5| null| 0.0| 0.0| 4.6|{1, [[[-74.002052...|{1, [[[-73.996152...|{5, [[[-80.0, 35....|
 CMT|2009-05-18 19:41:04|2009-05-18 19:52:22| 1| 1.4| -74.006536| 40.733116| null| -73.988564| 40.732145| 7.7| 0.0| null| 0.0| 0.0| 7.7|{1, [[[-74.006536...|{1, [[[-73.988564...|{5, [[[-80.0, 35....|
 CMT|2009-12-01 19:33:56|2009-12-01 19:47:17| 1| 2.8| -73.998658| 40.72502| null| -73.969251| 40.749782| 10.1| 1.0| 0.5| 0.0| 0.0| 11.6|{1, [[[-73.998658...|{1, [[[-73.969251...|{5, [[[-80.0, 35....|
 CMT|2009-04-01 15:29:07|2009-04-01 15:37:37| 1| 1.3| -74.007731| 40.725713| null| -73.993136| 40.732517| 6.5| 0.0| null| 0.0| 0.0| 6.5|{1, [[[-74.007731...|{1, [[[-73.993136...|{5, [[[-80.0, 35....|
 VTS|2010-02-13 10:11:00|2010-02-13 10:15:00| 6| 0.72| -73.996512| 40.723705| 1| -73.990087| 40.732678| 4.1| 0.0| 0.5| 0.0| 0.0| 4.6|{1, [[[-73.996512...|{1, [[[-73.990087...|{5, [[[-80.0, 35....|
 VTS|2009-02-26 17:26:00|2009-02-26 17:46:00| 2| 3.47| -74.006503| 40.72687| null| -73.97726| 40.761803| 13.3| 1.0| null| 0.0| 0.0| 14.3|{1, [[[-74.006503...|{1, [[[-73.97726,...|{5, [[[-80.0, 35....|
 VTS|2009-02-17 08:31:00|2009-02-17 08:53:00| 5| 4.83| -74.006813| 40.730107| null| -73.964982| 40.775298| 15.7| 0.0| null| 2.0| 0.0| 17.7|{1, [[[-74.006813...|{1, [[[-73.964982...|{5, [[[-80.0, 35....|
 VTS|2010-02-17 10:27:00|2010-02-17 10:41:00| 1| 3.03| -74.007658| 40.709973| 1| -74.00675| 40.747923| 10.1| 0.0| 0.5| 1.5| 0.0| 12.1|{1, [[[-74.007658...|{1, [[[-74.00675,...|{5, [[[-80.0, 35....|
 CMT|2009-12-04 00:56:47|2009-12-04 01:37:04| 1| 9.0| -74.005198| 40.729135| null| -73.93325| 40.689003| 26.1| 0.5| 0.5| 15.0| 0.0| 42.1|{1, [[[-74.005198...|{1, [[[-73.93325,...|{5, [[[-80.0, 35....|
 VTS|2009-05-20 17:02:00|2009-05-20 17:24:00| 5| 2.89| -73.990478| 40.729015| null| -73.976652| 40.762768| 12.9| 1.0| null| 3.0| 0.0| 16.9|{1, [[[-73.990478...|{1, [[[-73.976652...|{5, [[[-80.0, 35....|
 VTS|2010-05-21 14:11:00|2010-05-21 14:21:00| 1| 1.03| -74.014328| 40.704588| 1| -74.008928| 40.706917| 6.1| 0.0| 0.5| 0.0| 0.0| 6.6|{1, [[[-74.014328...|{1, [[[-74.008928...|{5, [[[-80.0, 35....|
 VTS|2009-12-08 17:10:00|2009-12-08 17:19:00| 1| 1.28| -73.994967| 40.725987| null| -73.977518| 40.723533| 6.5| 1.0| 0.5| 0.0| 0.0| 8.0|{1, [[[-73.994967...|{1, [[[-73.977518...|{5, [[[-80.0, 35....|
 CMT|2009-12-27 06:24:46|2009-12-27 06:44:15| 1| 6.0| -74.005439| 40.727577| null| -73.962271| 40.797077| 15.3| 0.0| 0.5| 3.16| 0.0| 18.96|{1, [[[-74.005439...|{1, [[[-

## Read from GeoJson, compute some basic geometry attributes

You've seen how Mosaic can create geometries from Spark native data types but it also provides functions to translate Well Known Text (WKT), Well Known Binary (WKB) and GeoJSON representations to Mosaic geometries.

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/shared_uploads/stuart.lynn@databricks.com/NYC_Taxi_Zones.geojson")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| geometry|
+-------------+-----------+--------+--------------------+--------------------+
 EWR| 1| 1| Newark Airport|{6, [[[-74.184452...|
 Queens| 2| 2| Jamaica Bay|{6, [[[-73.823375...|
 Bronx| 3| 3|Allerton/Pelham G...|{6, [[[-73.847926...|
 Manhattan| 4| 4| Alphabet City|{6, [[[-73.971774...|
Staten Island| 5| 5| Arden Heights|{6, [[[-74.174217...|
Staten Island| 6| 6|Arrochar/Fort Wad...|{6, [[[-74.063673...|
 Queens| 7| 7| Astoria|{6, [[[-73.904136...|
 Queens| 8| 8| Astoria Park|{6, [[[-73.923340...|
 Queens| 9| 9| Auburndale|{6, [[[-73.785024...|
 Manhattan| 24| 24| Bloomingdale|{6, [[[-73.959536...|
 Queens| 10| 10| Baisley Park|{6, [[[-73.783266...|
 Brooklyn| 11| 11| Bath Beach|{6, [[[-74.001098...|
 Manhattan| 12| 12| Battery Park|{6, [[[-74.015657...|
 Manhattan| 13| 13| Battery Park City|{6, [[[-74.012441...|
 Bronx| 18| 18| Bedford Park|{6, [[[-73.885139...|
 Brooklyn| 25| 25| Boerum Hill|{6, [[[-73.981552...|
 Brooklyn| 14| 14| Bay Ridge|{6, [[[-74.034073...|
 Queens| 15| 15|Bay Terrace/Fort ...|{6, [[[-73.777403...|
 Brooklyn| 22| 22| Bensonhurst West|{6, [[[-73.992549...|
Staten Island| 23| 23|Bloomfield/Emerso...|{6, [[[-74.195686...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

Mosaic provides a number of functions for extracting the properties of geometries. Here are some that are relevant to Polygon geometries:

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()

+--------------------+--------------------+--------------------+
 geometry| calculatedArea| calculatedLength|
+--------------------+--------------------+--------------------+
{6, [[[-74.184452...|7.823067885002562E-4| 0.1163574531886787|
{6, [[[-73.823375...|0.001422779097814599| 0.8431218810128789|
{6, [[[-73.847926...|3.144141568206508E-4| 0.08434110590105784|
{6, [[[-73.971774...| 7.94539194214528E-5| 0.03559982116907461|
{6, [[[-74.174217...|4.979574893632193E-4| 0.09214648985735088|
{6, [[[-74.063673...|4.046076598541714E-4| 0.12905753305870837|
{6, [[[-73.904136...|3.897879892739018...| 0.10741717112348065|
{6, [[[-73.923340...|2.658771690499072E-5|0.027590691195158792|
{6, [[[-73.785024...|3.384438031986125...| 0.0997840924710119|
{6, [[[-73.959536...|4.193691000471793E-5|0.034022318280805826|
{6, [[[-73.783266...|4.358238180810105...| 0.09983947941552027|
{6, [[[-74.001098...|2.029913532381591E-4| 0.07243934539937844|
{6, [[[-74.015657...|1.078595394871048...| 0.01759380786230351|
{6, [[[-74.012441...|4.874002713725957E-5| 0.04726120195723124|
{6, [[[-73.885139...|1.488501639473639E-4| 0.06979954985700013|
{6, [[[-73.981552...|1.241682673558662...| 0.04714581993191898|
{6, [[[-74.034073...|6.636590898190788E-4| 0.13622530962992194|
{6, [[[-73.777403...|4.588438269978550...| 0.1295148763926726|
{6, [[[-73.992549...|4.613649657988681...| 0.11844001178496774|
{6, [[[-74.195686...|0.002090822778761513| 0.3007595308122533|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
geoJsonDF.count()

Out[40]: 263

## Example point-in-poly with indexing

Mosaic has built-in support for the popular spatial indexing library, H3. The user has access to functions for generating point indices and the sets of indices covering polygons, allowing point-in-polygon joins to be transformed into deterministic SQL joins.

In [0]:
from mosaic import point_index

trips_with_geom = (
  trips
  .withColumn("pickup_h3", point_index(lng="pickup_longitude", lat="pickup_latitude", resolution=lit(10)))
  .withColumn("dropoff_h3", point_index(lng="dropoff_longitude", lat="dropoff_latitude", resolution=lit(10)))
)

trips_with_geom.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
 VTS|2009-04-23 22:34:00|2009-04-23 22:52:00| 1| 4.45| -73.994352| 40.70314| null| -73.998247| 40.745143| 13.7| 0.5| null| 0.0| 0.0| 14.2|{1, [[[-73.994352...|{1, [[[-73.998247...|{5, [[[-80.0, 35....|622236750726889471|622236723430260735|
 VTS|2009-07-15 18:02:00|2009-07-15 18:15:00| 5| 2.65| -74.000485| 40.732423| null| -73.970637| 40.75256| 9.7| 1.0| null| 0.0| 0.0| 10.7|{1, [[[-74.000485...|{1, [[[-73.970637...|{5, [[[-80.0, 35....|622236750719713279|622236723495075839|
 CMT|2010-02-11 19:50:32|2010-02-11 19:58:20| 1| 1.3| -74.015856| 40.711017| 1| -74.007158| 40.726024| 6.9| 1.0| 0.5| 0.0| 0.0| 8.4|{1, [[[-74.015856...|{1, [[[-74.007158...|{5, [[[-80.0, 35....|622236750652506111|622236750711259135|
 VTS|2009-09-17 05:51:00|2009-09-17 05:54:00| 1| 0.78| -74.002052| 40.72806| null| -73.996152| 40.737065| 4.1| 0.5| null| 0.0| 0.0| 4.6|{1, [[[-74.002052...|{1, [[[-73.996152...|{5, [[[-80.0, 35....|622236750718468095|622236723432357887|
 CMT|2009-05-18 19:41:04|2009-05-18 19:52:22| 1| 1.4| -74.006536| 40.733116| null| -73.988564| 40.732145| 7.7| 0.0| null| 0.0| 0.0| 7.7|{1, [[[-74.006536...|{1, [[[-73.988564...|{5, [[[-80.0, 35....|622236750719778815|622236723428818943|
 CMT|2009-12-01 19:33:56|2009-12-01 19:47:17| 1| 2.8| -73.998658| 40.72502| null| -73.969251| 40.749782| 10.1| 1.0| 0.5| 0.0| 0.0| 11.6|{1, [[[-73.998658...|{1, [[[-73.969251...|{5, [[[-80.0, 35....|622236750706835455|622236723495436287|
 CMT|2009-04-01 15:29:07|2009-04-01 15:37:37| 1| 1.3| -74.007731| 40.725713| null| -73.993136| 40.732517| 6.5| 0.0| null| 0.0| 0.0| 6.5|{1, [[[-74.007731...|{1, [[[-73.993136...|{5, [[[-80.0, 35....|622236750711128063|622236723432456191|
 VTS|2010-02-13 10:11:00|2010-02-13 10:15:00| 6| 0.72| -73.996512| 40.723705| 1| -73.990087| 40.732678| 4.1| 0.0| 0.5| 0.0| 0.0| 4.6|{1, [[[-73.996512...|{1, [[[-73.990087...|{5, [[[-80.0, 35....|622236750707064831|622236723428589567|
 VTS|2009-02-26 17:26:00|2009-02-26 17:46:00| 2| 3.47| -74.006503| 40.72687| null| -73.97726| 40.761803| 13.3| 1.0| null| 0.0| 0.0| 14.3|{1, [[[-74.006503...|{1, [[[-73.97726,...|{5, [[[-80.0, 35....|622236750719025151|622236723493699583|
 VTS|2009-02-17 08:31:00|2009-02-17 08:53:00| 5| 4.83| -74.006813| 40.730107| null| -73.964982| 40.775298| 15.7| 0.0| null| 2.0| 0.0| 17.7|{1, [[[-74.006813...|{1, [[[-73.964982...|{5, [[[-80.0, 35....|622236750718894079|622236722208505855|
 VTS|2010-02-17 10:27:00|2010-02-17 10:41:00| 1| 3.03| -74.007658| 40.709973| 1| -74.00675| 40.747923| 10.1| 0.0| 0.5| 1.5| 0.0| 12.1|{1, [[[-74.007658...|{1, [[[-74.00675,...|{5, [[[-80.0, 35....|622236750646902783|622236750530543615|
 CMT|2009-12-04 00:56:47|2009-12-04 01:37:04| 1| 9.0| -74.005198| 40.729135| null| -73.93325| 40.689003| 26.1| 0.5| 0.5| 15.0| 0.0| 42.1|{1, [[[-74.005198...|{1, [[[-73.93325,...|{5, [[[-80.0, 35....|622236750719123455|622236723535740927|
 VTS|2009-05-20 17:02:00|2009-05-20 17:24:00| 5| 2.89| -73.990478| 40.729015| null| -73.976652| 40.762768| 12.9| 1.0| null| 3.0| 0.0| 16.9|{1, [[[-73.990478...|{1, [[[-73.976652...|{5, [[[-80.0, 35....|62223675071

In [0]:
from mosaic import polyfill

neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", explode(polyfill("geometry", lit(10))).alias("h3"))
  .drop("geometry")
)

neighbourhoods.show()

+-------+-----------+--------+--------------------+------------------+
borough|location_id|objectid| zone| h3|
+-------+-----------+--------+--------------------+------------------+
 Queens| 207| 207|Saint Michaels Ce...|622236723991183359|
 Queens| 207| 207|Saint Michaels Ce...|622236723950878719|
 Queens| 207| 207|Saint Michaels Ce...|622236723989315583|
 Queens| 207| 207|Saint Michaels Ce...|622236723991216127|
 Queens| 207| 207|Saint Michaels Ce...|622236723988201471|
 Queens| 207| 207|Saint Michaels Ce...|622236723991248895|
 Queens| 207| 207|Saint Michaels Ce...|622236723990134783|
 Queens| 207| 207|Saint Michaels Ce...|622236723990167551|
 Queens| 207| 207|Saint Michaels Ce...|622236723950223359|
 Queens| 207| 207|Saint Michaels Ce...|622236723988299775|
 Queens| 207| 207|Saint Michaels Ce...|622236723951009791|
 Queens| 207| 207|Saint Michaels Ce...|622236723950256127|
 Queens| 207| 207|Saint Michaels Ce...|622236723988332543|
 Queens| 207| 207|Saint Michaels Ce...|622236723951042559|
 Queens| 207| 207|Saint Michaels Ce...|622236723991379967|
 Queens| 207| 207|Saint Michaels Ce...|622236723950682111|
 Queens| 207| 207|Saint Michaels Ce...|622236723950714879|
 Queens| 207| 207|Saint Michaels Ce...|622236723991052287|
 Queens| 207| 207|Saint Michaels Ce...|622236723950747647|
 Queens| 207| 207|Saint Michaels Ce...|622236723951140863|
+-------+-----------+--------+--------------------+------------------+
only showing top 20 rows

In [0]:
joined_df = trips_with_geom.alias("t").join(neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
joined_df.count()

Out[43]: 4936537

## Mosaic spatial join optimizations

Mosaic provides easy access to the optimized spatial join technique described in [this](https://databricks.com/blog/2021/10/11/efficient-point-in-polygon-joins-via-pyspark-and-bng-geospatial-indexing.html) blog post.

In [0]:
from mosaic import mosaic_explode

mosaic_neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", mosaic_explode("geometry", lit(10)))
  .drop("geometry")
)

mosaic_neighbourhoods.show()

+-------+-----------+--------+--------------------+-------+------------------+--------------------+
borough|location_id|objectid| zone|is_core| h3| wkb|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950682111| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950845951| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951009791| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988332543| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950780415| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991085055| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991248895| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950714879| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991183359| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988299775| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950747647| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991052287| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991216127| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991379967| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951140863| null|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723990167551|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723991117823|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950288895|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723988365311|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950878719|[01 03 00 00 00 0...|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
only showing top 20 rows

Mosaic also includes a convenience function for displaying dataframes with geometry columns.

In [0]:
from mosaic import displayMosaic
displayMosaic(mosaic_neighbourhoods)

borough,location_id,objectid,zone,is_core,h3,wkb
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723950682111,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723950845951,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723951009791,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723988332543,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723950780415,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723991085055,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723991248895,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723950714879,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723991183359,null
Queens,207,207,Saint Michaels Cemetery/Woodside,true,622236723988299775,null


This also extends to plotting maps inside the notebook using the kepler.gl visualisation library using a notebook magic `%%mosaic_kepler`.

In [0]:
from mosaic import st_aswkt
(
  mosaic_neighbourhoods
  .select(st_aswkt(col("wkb")).alias("wkt"), col("h3"))
#   .withColumn("h3_id", col("h3"))
#   .withColumn("wkt", st_aswkt("wkb"))
#   .drop("wkb")
).createOrReplaceTempView("kepler_df")

In [0]:
%%mosaic_kepler
"kepler_df" "h3" "h3"

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

<!doctype html> Kepler.gl

Now the two datasets can be joined first on H3 index, with any false positives removed through a contains filter on a much simpler geometry.

In [0]:
mosaic_joined_df = (
  trips_with_geom.alias("t")
  .join(mosaic_neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
  .where(
    ~col("is_core") | 
    st_contains("wkb", "pickup_geom")
  )
)

mosaic_joined_df.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3| borough|location_id|objectid| zone|is_core| h3| wkb|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
 VTS|2009-04-23 22:34:00|2009-04-23 22:52:00| 1| 4.45| -73.994352| 40.70314| null| -73.998247| 40.745143| 13.7| 0.5| null| 0.0| 0.0| 14.2|{1, [[[-73.994352...|{1, [[[-73.998247...|{5, [[[-80.0, 35....|622236750726889471|622236723430260735| Brooklyn| 66| 66| DUMBO/Vinegar Hill| false|622236750726889471|[01 03 00 00 00 0...|
 VTS|2009-07-15 18:02:00|2009-07-15 18:15:00| 5| 2.65| -74.000485| 40.732423| null| -73.970637| 40.75256| 9.7| 1.0| null| 0.0| 0.0| 10.7|{1, [[[-74.000485...|{1, [[[-73.970637...|{5, [[[-80.0, 35....|622236750719713279|622236723495075839|Manhattan| 249| 249| West Village| false|622236750719713279|[01 03 00 00 00 0...|
 VTS|2009-07-15 18:02:00|2009-07-15 18:15:00| 5| 2.65| -74.000485| 40.732423| null| -73.970637| 40.75256| 9.7| 1.0| null| 0.0| 0.0| 10.7|{1, [[[-74.000485...|{1, [[[-73.970637...|{5, [[[-80.0, 35....|622236750719713279|622236723495075839|Manhattan| 114| 114|Greenwich Village...| false|622236750719713279|[01 03 00 00 00 0...|
 VTS|2009-09-17 05:51:00|2009-09-17 05:54:00| 1| 0.78| -74.002052| 40.72806| null| -73.996152| 40.737065| 4.1| 0.5| null| 0.0| 0.0| 4.6|{1, [[[-74.002052...|{1, [[[-73.996152...|{5, [[[-80.0, 35....|622236750718468095|622236723432357887|Manhattan| 249| 249| West Village| false|622236750718468095|[01 03 00 00 00 0...|
 VTS|2009-09-17 05:51:00|2009-09-17 05:54:00| 1| 0.78| -74.002052| 40.72806| null| -73.996152| 40.737065| 4.1| 0.5| null| 0.0| 0.0| 4.6|{1, [[[-74.002052...|{1, [[[-73.996152...|{5, [[[-80.0, 35....|622236750718468095|622236723432357887|Manhattan| 125| 125| Hudson Sq| false|622236750718468095|[01 03 00 00 00 0...|
 VTS|2009-09-17 05:51:00|2009-09-17 05:54:00| 1| 0.78| -74.002052| 40.72806| null| -73.996152| 40.737065| 4.1| 0.5| null| 0.0| 0.0| 4.6|{1, [[[-74.002052...|{1, [[[-73.996152...|{5, [[[-80.0, 35....|622236750718468095|622236723432357887|Manhattan| 211| 211| SoHo| false|622236750718468095|[01 03 00 00 00 0...|
 VTS|2009-09-17 05:51:00|2009-09-17 05:54:00| 1| 0.78| -74.002052| 40.72806| null| -73.996152| 40.737065| 4.1| 0.5| null| 0.0| 0.0| 4.6|{1, [[[-74.002052...|{1, [[[-73.996152...|{5, [[[-80.0, 35....|622236750718468095|622236723432357887|Manhattan| 114| 114|Greenwich Village...| false|622236750718468095|[01 03 00 00 00 0...|
 CMT|2009-05-18 19:41:04|2009-05-18 19:52:22| 1| 1.4| -74.006536| 40.733116| null| -73.988564| 40.732145| 7.7| 0.0| null| 0.0| 0.0| 7.7|{1, [[[-74.006536...|{1, [[[-73.988564...|{5, [[[-80.0, 35....|622236750719778815|622236723428818943|Manhattan| 249| 249| West Village| false|622236750719778815|[01 03 00 00 00 0...|
 CMT|2009-05-18 19:41:04|2009-05-18 19:52:22| 1| 1.4| -74.006536| 40.733116| null| -73.988564| 40.732145| 7.7| 0.0| null| 0.0| 0.0| 7.7|{1, [[[-74.006536...|{1, [[[-73.988564...|{5, [[[-80.0, 35....|622236750719778815|622236723428818943|Manhattan| 158|